https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/hela-sverige-ska-fungera--kristdemokraternas_H9024221

In [4]:
import pandas as pd

In [1]:
html_url = 'https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/hela-sverige-ska-fungera--kristdemokraternas_H9024221/html#_Toc84341143'

In [5]:
tables = pd.read_html(html_url)

In [6]:
len(tables)

15

In [8]:
df = tables[10]

In [10]:
df.columns = df.iloc[0]
df = df.iloc[1:]

In [12]:
df = df.replace('±0', '0')

In [14]:
df.to_csv('kristdemokraterna-budgetmotion-2022.csv', index=False)